<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_SeongGwan/split_coding_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

plt.rc('font', family='NanumBarunGothic')

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


In [10]:
df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/ops_data_2021.csv', encoding='CP949')
df['day'] = df['day'].apply(lambda x: int(x.replace('-', '')))

df.head()

,name,day,vs,result,bat_order,position,start_member,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA
0,강백호,404,한화,W 3:2,4,1B,1,3,0,2,0,0,0,2,0,2,0,1,0,0,0,0,0,0,0.667,NaN,0.667,1.417,16,1.14,0.89,0.122
1,강백호,406,LG,L 2:3,4,1B,1,4,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.286,NaN,0.286,0.661,14,2.07,-1.00,-0.183
2,강백호,407,LG,W 7:3,4,1B,1,4,0,3,0,0,0,3,1,0,0,1,0,0,0,0,0,0,0.455,NaN,0.455,0.993,17,1.43,1.75,0.330
3,강백호,408,LG,L 3:7,4,1B,1,4,0,2,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0.467,NaN,0.467,0.996,17,0.89,-0.76,-0.102
4,강백호,409,@삼성,L 5:7,4,1B,1,5,0,3,0,1,0,5,3,0,0,0,0,0,1,1,0,0,0.500,NaN,0.600,1.145,21,0.94,1.88,-0.122


## Spliter Usage

- 변수

    1. train_num : 학습에 사용할 경기 수 (order에만 사용됨) *default: 50*
    2. pred_num : 예측할 경기 수 *default: 30*
    3. seed : 랜덤 시드 (random에만 사용됨) *default: 1004*
    5. start_date : order 분할에서 초기값으로 사용할 날짜 (문자형) *default: '0301'*
    6. except_print : 경기수가 모자라 삭제되는 선수들 목록 프린트 여부 *default: False*


- 사용 예시

        spliter1 = Spliter()
        spliter1.SetData(df)

        order_2021 = spliter1.order()
        random_2021 = spliter1.random()

        --------------------------------------------------

        spliter2 = Spliter()
        spliter2.SetData(df2)

        spliter2.train_num = 30
        spliter2.pred_num = 10
        spliter2.start_date = '0501'
        
        order_2020 = spliter2.order()
        random_2020 = spliter2.random()


In [80]:
class Spliter:

    def __init__(self):
        self.train_num = 50
        self.pred_num = 30
        self.seed = 1004
        self.start_date = '0301'
        self.df = ''
        self.except_print = False

    # 데이터셋 정하는 함수
    def SetData(self,dataname):
        self.df = dataname
    
    def GetData(self):
        return self.df

    # 연속적 분할 함수
    def order(self):

        # 해당 시즌 타자 목록 (name -> pcode)
        hitter_list = list(set(self.df['name']))

        # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외) 
        target_col = list(range(7, 24))
        target_col.append(28)
        target_col.append(31)

        # start_date 이후의 경기만 추출
        later_df = self.df[ self.df['day']  >= int(self.start_date) ]

        # 결과값 저장할 데이터프레임 생성
        split_result = pd.DataFrame()
        
        # 타자 한명씩 추출
        for player in hitter_list:

            # name -> pcode
            pl_data = later_df[later_df['name'] == player]

            # 경기 수가 train_num+pred_num보다 적으면 삭제 : 어떤 선수 제외되는지 출력 opt
            if len(pl_data) < self.train_num+self.pred_num:
                if self.except_print == True:
                    print('{}은/는 경기 수가 {}보다 작기 때문에 삭제'.format(player, self.train_num+self.pred_num))
                    pass
                else:
                    pass

            else:
                # 날짜순으로 정렬
                pl_data = pl_data.sort_values('day').reset_index(drop=True)

                # train_num만큼의 경기를 tr_data에 저장
                tr_data = pl_data[:self.train_num]
                # 그 이후 pred_num만큼의 경기를 ts_data에 저장
                ts_data = pl_data[self.train_num:self.train_num+self.pred_num]
                
                # tr_data의 타겟 칼럼을 모두 합해서 저장
                sum_data = tr_data.iloc[:, target_col].sum()
                # 어떤 선수인지 저장 (name -> pcode)
                sum_data['name'] = player

                # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
                ts_sum = ts_data.iloc[:, target_col].sum()
                sum_data['avg_label'] = ts_sum['hit'] / ts_sum['ab']
                sum_data['slg_label'] = ts_sum['tb'] / ts_sum['ab']
                sum_data['obp_label'] = (ts_sum['hit'] + ts_sum['bb'] + ts_sum['hpp']) / (ts_sum['ab'] +  ts_sum['bb'] + ts_sum['hpp'] + ts_sum['sf'])
                sum_data['ops_label'] = sum_data['slg_label'] + sum_data['obp_label']

                # Series 데이터를 DataFrame 형식으로 변환
                sum_result = sum_data.to_frame().transpose()
                
                # 이 선수 데이터를 전체 데이터셋에 추가
                split_result = pd.concat([split_result, sum_result])
        
        # 데이터셋 인덱스 초기화
        split_result = split_result.reset_index(drop=True)
        
        return split_result


    # 랜덤 마스킹 분할 함수
    def random(self):
        
        # 해당 시즌 타자 목록 (name -> pcode)
        hitter_list = list(set(self.df['name']))

        # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외) 
        target_col = list(range(7, 24))
        target_col.append(28)
        target_col.append(31)
        
        # 결과값 저장할 데이터프레임 생성
        split_result = pd.DataFrame()
        
        # 타자 한명씩 추출
        for player in hitter_list:
            pl_data = self.df[self.df['name'] == player].reset_index(drop=True)

            # 경기 수가 pred_num의 2배보다 적으면 삭제 (학습할 경기 수 >= 예측할 경기 수)
            if len(pl_data) < self.pred_num*2:
                if self.except_print == True:
                    print('{}은/는 경기 수가 {}보다 작기 때문에 삭제'.format(player, self.pred_num*2))
                    pass
                else:
                    pass

            else:
                # 날짜순으로 정렬
                pl_data = pl_data.sort_values('day').reset_index(drop=True)

                # pred_num을 뺀 만큼의 경기를 랜덤으로 추출해서 tr_data에 저장
                tr_data = pl_data.sample(n=len(pl_data) - self.pred_num, random_state=self.seed)
                # 추출된 인덱스 저장
                tr_idx = list(tr_data.index)

                # 마스킹(제외)된 경기들을 ts_data에 저장
                ts_data = pl_data.drop(tr_idx)

                # tr_data의 타겟 칼럼을 모두 합해서 저장
                sum_data = tr_data.iloc[:, target_col].sum()
                # 어떤 선수인지 저장 (name -> pcode)
                sum_data['name'] = player
                
                # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
                ts_sum = ts_data.iloc[:, target_col].sum()
                sum_data['avg_label'] = ts_sum['hit'] / ts_sum['ab']
                sum_data['slg_label'] = ts_sum['tb'] / ts_sum['ab']
                sum_data['obp_label'] = (ts_sum['hit'] + ts_sum['bb'] + ts_sum['hpp']) / (ts_sum['ab'] +  ts_sum['bb'] + ts_sum['hpp'] + ts_sum['sf'])
                sum_data['ops_label'] = sum_data['slg_label'] + sum_data['obp_label']

                # Series 데이터를 DataFrame 형식으로 변환
                sum_result = sum_data.to_frame().transpose()

                # 이 선수 데이터를 전체 데이터셋에 추가
                split_result = pd.concat([split_result, sum_result])
        
        # 데이터셋 인덱스 초기화
        split_result = split_result.reset_index(drop=True)
        
        return split_result

In [90]:
spliter = Spliter()
spliter.SetData(df)

spliter.order()

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,pitch,WPA,name,avg_label,slg_label,obp_label,ops_label
0,180,31,60,13,0,2,79,29,5,0,21,3,0,24,7,0,2,766,0.344,박건우,0.315315,0.423423,0.388889,0.812312
1,122,23,30,5,0,1,38,12,11,2,11,0,0,15,0,5,3,507,-0.261,김지찬,0.279412,0.279412,0.3,0.579412
2,162,33,46,11,0,4,69,29,7,3,28,1,1,24,4,0,3,751,0.39,마차도,0.256637,0.283186,0.338462,0.621647
3,187,27,59,11,0,2,76,27,2,1,30,3,0,22,4,1,3,854,0.613,전준우,0.344538,0.504202,0.407407,0.911609
4,170,18,45,9,0,8,78,25,0,0,19,6,1,46,1,0,3,788,-0.227,한유섬,0.258065,0.548387,0.375,0.923387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,190,27,78,11,1,7,112,47,4,2,32,0,3,29,4,0,3,952,2.808,강백호,0.371134,0.556701,0.519685,1.07639
58,108,17,27,4,1,5,48,11,1,3,6,4,0,25,1,3,1,486,-1.15,김강민,0.229508,0.42623,0.364865,0.791094
59,124,24,26,6,0,4,44,12,2,0,6,0,0,28,0,4,0,474,-0.402,정진기,0.217391,0.347826,0.217391,0.565217
60,191,35,54,10,1,4,78,26,4,2,23,3,0,46,1,0,0,860,-0.034,하주석,0.243478,0.33913,0.314961,0.654091


In [92]:
spliter.pred_num = 20
spliter.seed = 0
spliter.except_print = True
spliter.random()

김태훈은/는 경기 수가 40보다 작기 때문에 삭제
김도환은/는 경기 수가 40보다 작기 때문에 삭제
이현석은/는 경기 수가 40보다 작기 때문에 삭제
오명진은/는 경기 수가 40보다 작기 때문에 삭제
김윤수은/는 경기 수가 40보다 작기 때문에 삭제
김재현은/는 경기 수가 40보다 작기 때문에 삭제
민병헌은/는 경기 수가 40보다 작기 때문에 삭제
최보성은/는 경기 수가 40보다 작기 때문에 삭제
최정용은/는 경기 수가 40보다 작기 때문에 삭제
김응민은/는 경기 수가 40보다 작기 때문에 삭제
허관회은/는 경기 수가 40보다 작기 때문에 삭제
변상권은/는 경기 수가 40보다 작기 때문에 삭제
예진원은/는 경기 수가 40보다 작기 때문에 삭제
김호은은/는 경기 수가 40보다 작기 때문에 삭제
김창평은/는 경기 수가 40보다 작기 때문에 삭제
김주현은/는 경기 수가 40보다 작기 때문에 삭제
최항은/는 경기 수가 40보다 작기 때문에 삭제
김건형은/는 경기 수가 40보다 작기 때문에 삭제
강태율은/는 경기 수가 40보다 작기 때문에 삭제
허정협은/는 경기 수가 40보다 작기 때문에 삭제
백동훈은/는 경기 수가 40보다 작기 때문에 삭제
박정우은/는 경기 수가 40보다 작기 때문에 삭제
이진영은/는 경기 수가 40보다 작기 때문에 삭제
박주홍은/는 경기 수가 40보다 작기 때문에 삭제
이호연은/는 경기 수가 40보다 작기 때문에 삭제
강상원은/는 경기 수가 40보다 작기 때문에 삭제
유원상은/는 경기 수가 40보다 작기 때문에 삭제
이창재은/는 경기 수가 40보다 작기 때문에 삭제
강로한은/는 경기 수가 40보다 작기 때문에 삭제
최정민은/는 경기 수가 40보다 작기 때문에 삭제
유장혁은/는 경기 수가 40보다 작기 때문에 삭제
황경태은/는 경기 수가 40보다 작기 때문에 삭제
노태형은/는 경기 수가 40보다 작기 때문에 삭제
모창민은/는 경기 수가 40보다 작기 때문에 삭제
보어은/는 경기 수가 40보다 작기 때문에 삭제
남호은/는 경기 수가 40보다 작기 때문

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,pitch,WPA,name,avg_label,slg_label,obp_label,ops_label
0,87,10,15,2,0,1,20,10,2,1,8,1,0,31,2,2,0,409,-0.448,이학주,0.283019,0.490566,0.333333,0.823899
1,133,15,33,6,1,6,59,20,0,1,21,0,0,23,4,1,2,666,0.483,문보경,0.216667,0.366667,0.381579,0.748246
2,45,7,12,0,1,0,14,1,2,0,11,1,0,10,1,1,0,249,0.046,김민혁,0.266667,0.311111,0.3125,0.623611
3,180,23,53,9,0,6,80,31,1,0,12,1,0,35,3,0,1,747,0.317,힐리,0.15942,0.26087,0.216216,0.477086
4,225,33,76,19,0,2,101,31,5,0,22,4,1,32,10,0,2,922,0.55,박건우,0.285714,0.371429,0.392857,0.764286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,289,47,79,17,3,4,114,30,10,3,32,6,0,76,2,3,2,1354,-0.714,하주석,0.298701,0.441558,0.357143,0.798701
135,72,7,13,2,0,4,27,11,1,2,4,1,0,33,2,1,1,327,-0.75,조한민,0.214286,0.452381,0.333333,0.785714
136,103,15,26,4,0,1,33,9,1,0,5,3,0,27,3,0,3,456,-0.756,안재석,0.270833,0.458333,0.326923,0.785256
137,27,5,6,1,0,1,10,5,1,0,4,2,0,6,1,0,0,135,-0.123,도태훈,0.142857,0.142857,0.172414,0.315271
